# Consensus Analysis

The objective for this analysis is to generate MinION consensus sequences that are 100% identical to either:
A) Their reference sequence (only applicable for the lab strain and plasmids)
B) Their MiSeq counterpart

## Lab Strain Consensus Analysis
For the lab strains, reads were mapped to the references provided by Bin for each subtype.

In [1]:
# MinION data
bwa index ont2d all-minion-flu-cds-only.fasta
bwa mem -t 40 -x ont2d all-minion-flu-cds-only.fasta flu_4_rebasecalled.fastq |\
samtools view -bS - | \
samtools sort - flu_4_rebasecalled.all-flu-standard-minion && \
samtools index flu_4_rebasecalled.all-flu-standard-minion.bam

# PacBio Data
blasr Elodie_sidebar_reads_of_insert.fastq all-minion-flu-cds-only.fasta \
-sam -clipping soft -out pacbio-quad-cds-only.bam -nproc 40 -bestn 1

# MiSeq Data
bowtie2-build all-minion-flu-cds-only.fasta all-minion-flu-cds-only.fasta
bowtie2 -p 40 --very-sensitive-local -x all-minion-flu-cds-only.fasta

# Ion Data
bowtie2 -p 40 --very-sensitive-local -x all-minion-flu-cds-only.fasta

# To generate consensus sequences:
for i in $(samtools idxstats flu_4_rebasecalled.all-flu-standard-minion.bam | cut -f1 | grep -v '*')
do
    for x in $(ls *bam | perl -pe 's/\.bam//g')
    do
        python bamfile_consensus_generate.py $x.bam $i >> $x-$i.consensus.fasta
    done
done

# To compare the consensus sequences:
for i in $(samtools idxstats flu_4_rebasecalled.all-flu-standard-minion.bam | cut -f1 | grep -v '*')
do

    grep -A1 -h --no-group-separator $i miseq-quad-cds-only-$i.consensus.fasta \
    flu_4_rebasecalled.all-flu-standard-minion-$i.consensus.fasta > miseq-minion-$i.consensus.fasta\
    && python pairwise-pid.py miseq-minion-$i.consensus.fasta fasta > miseq-minion-$i.consensus.variants.txt

done


[bwa_index] fail to open file 'ont2d' : No such file or directory
[bam_sort] Use -T PREFIX / -o FILE to specify temporary and final output files
Usage: samtools sort [options...] [in.bam]
Options:
  -l INT     Set compression level, from 0 (uncompressed) to 9 (best)
  -m INT     Set maximum memory per thread; suffix K/M/G recognized [768M]
  -n         Sort by read name
  -o FILE    Write final output to FILE rather than standard output
  -T PREFIX  Write temporary files to PREFIX.nnnn.bam
  -@, --threads INT
             Set number of sorting and compression threads [1]
      --input-fmt-option OPT[=VAL]
               Specify a single input file format option in the form
               of OPTION or OPTION=VALUE
  -O, --output-fmt FORMAT[,OPT[=VAL]]...
               Specify output format (SAM, BAM, CRAM)
      --output-fmt-option OPT[=VAL]
               Specify a single output file format option in the form
               of OPTION or OPTION=VALUE
      --reference FILE
            

All but FluB-PB1 were resolved with this workflow as there was a suspected sequence-specific error in the PB1 segment. This was corrected with the latest 11-09 run.

## Clinical Strain Consensus Analysis

### First attempt
The first run of the consensus analysis was done in the same fashion with the same reference as the lab strains: map the raw reads to the consensus sequences to try and obtain a consensus sequence that is 100% identical to its reference. However, it didn't turn out so good. The main problem was that the workflow was contingent upon a known reference which limits the purpose of using this for diagnostics. It also did not match up with the MiSeq data which suggests something went awry.

### Second attempt
The second attempt was actually emulate a simple clinical analysis that would be used for diagnostics while using the MiSeq for validation.

The first step is to blast the MinION reads to a Flu specific database from [FluDB](http://www.fludb.org/brc/influenza_sequence_search_segment_display.spg?method=ShowCleanSearch&decorator=influenza). This is used to improve runtime by only querying Flu and, further, the subtypes we know are present. The original implementation of this algorithm was limiting the possible strains to the years 2014-2015 since that's the timeline our samples were obtained.

<img src="flu-db-search-criteria.png">

In [ ]:
makeblastdb -dbtype nucl -in all-flu-2014-2015.fasta

# also did this for the 8-13 and 11-9 runs
blastn \
-db all-flu-2014-2015.fasta \
-query flu-6-4-rebasecalled.pass.fasta \
-outfmt 5 \
-out flu-6-4-rebasecalled.pass.all-flu-2014-15.blastn.xml \
-evalue 0.00005\
-num_threads 40 \
-culling_limit 2 \
-max_target_seqs 1

python read-fludb-blastxml.py flu-6-4-rebasecalled.pass.all-flu-2014-15.blastn.xml > flu-6-4-rebasecalled.pass.all-flu-2014-15.blast.seg-counts.tsv

## 2-11

[ ] Align 11-09 data to MiSeq FluB-PB2 consensus and see if the discrepancy at position 1545 is resolved.

MinION Flu Analysis

For Elodies Presentation:

- Compare metrichor version output
    - compare sequencing kit output
- can use MinION for identification of mixed infections
    - subtypes are easily separated
    - FluB-PB1 clinical and standard were identifiable after being run through the flowcell
    - Can easily distinguish between subtypes
    - Maybe distinguish between H3N2-70 and H3N2-90?
- SNV calling + haplotypes
    - L1 Norm of only MiSeq variants compared to corresponding MinION positions
    - L1 Norm of all MiSeq position compared to all MinION positions
- Error correction for haplotyping
    - I already have a working version of the page rank algorithm to find linked variants

To Do:

[ ] Assume mixed infection, take top 10 hits per segment, and try to generate consensus

- FluB Yamagata : http://www.ncbi.nlm.nih.gov/nuccore/937172197
- FluB Victoria: http://www.ncbi.nlm.nih.gov/nucleotide/937171812

- -----

[x] Curate flu specific database with 2014-2015 (us only?) to query reads to obtain reference.

- Do this on a per segment basis
- One thing to note is that there were a lot of reads assigned to the Egypt strain and not florida - maybe because of the flanking regions being present?
    - Make the database cds only

     [ ] Blast H3N2-70 and H3N2-90 separately
     [ ] Use latest run to resolve FluB mutations

[ ] Blast MinION reads for the clinical samples and use the hits for a reference

- See if the mixed infection data can distinguish the different subtypes
    - blastn didn’t do so hot. Only 1009 reads were identified as ssRNA viruses and the subtypes were all over the place.
        - blastn -num_threads 40 -db /data1/share/db/nt/nt -query all-6-4_8-13_11-09.pass.fasta -out all-6-4_8-13_11-09.pass.blastn.xml -outfmt 5 -max_target_seqs 1 -culling_limit 2 -evalue 0.00005
- Use the references to make a consensus instead of the MiSeq consensus to mimic the practical application of the MinION
- Examine run by run to see if the h3n2 data (and lab strains) are interfering with the blast results.

[ ] Top variants for each platform and look at the overlap between them

[ ] Align to Flu Plasmids

- Definitely in the 11-09 run

[x] IGV snapshot of quads study of the new data while keeping old data. Look at H1N1

[x] Coverage plots of each sequencing run with consensus variants and deletions in it

[ ] Variant analysis if possible (a big plus if the result is similar to the MiSeq data).
     [x] Top SNV comparison between 11-09 data and MiSeq (map MiSeq and MinION data to the MiSeq consensus sequences and plot the % of the consensus nucleotide. Do this with plasmid data
     [x] Percent reference base
     [ ] Percent snv
     [ ] L1 norm (manhattan distance). This is a way to obtain variance between the MiSeq and MinION variance data.
          [ ] MiSeq variant sites (from Tim's snplist script) compared to those positions in MinION (using rebasecalled data):
     [ ] Look to see if coverage has an affect - overlay depth?
               [ ] 4-20/4-21
               [x] 6-4/8-13
               [ ] 11-09
          [ ] All sites from both platforms combined
               [ ] 4-20/4-21
               [x] 6-4/8-13
               [ ] 11-09

[ ] Comparison of old sequencing kit (SK005) vs new sequencing kit (SK006) (plot difference of L1Norm)

[x] H1N1 4-20/4-21 vs 11-09

[x] FluB 6-4/8-13 vs 11-09

[x] FluB-PB1 4-20/4-21 vs 11-09

[ ] Compare to MiSeq
     [ ] MiSeq vs old kit
     [ ] MiSeq vs new kit

- Why is the H1N1 NS segment so bad?

[ ] Comparison of old metrichor results to new metrichor results (not applicable to run 11-09)

[ ] Write script to compare sequence content

    - Quality score content?

[ ] 4-20/4-21

[x] 6-4

[x] 8-13

- Need to compare in timeline?

[x] See if H3N2-70 is distinguishable from H3N2-90 when mixed (use rebasecalled data)
     [x] Alignment
     [x] Table of variants between the two
     [ ] See if there’s any carry over with the H3N2 variants in the alignments

- The 6-4 and 8-13 data were merged and H3N2-70 and H3N2-90 were resolved at 100% consensus accuracy (minus the variants that were corrected by the 11-09 run)
- Data were aligned to all clinical strains (H3N2-70, H3N2-90, FluB, and pH1N1)
- Need figure for this

    - Maybe see if they cluster together?
    - Coverage plot for now?

[x] All the 3 lab reference strains and all the 3 clinical samples at 100% correct at consensus level, compared to the MiSeq data (minimal requirement for the paper)

[x] rebasecalled all of the reads

[x] H1N1 and FluB-PB1 lab standards to be resequenced.

[x] FluB-PB1 clinical to be resequenced

[x] 11-9 data was 100% for each sequence

[x] Compare FluB clinical consensus and lab standard to see if they’re distinct enough to be run on the same flow cell.

Interesting Things:

- Plot quality scores in FluB-PB1 mutation to see if that’s something
    - suspected SSE was found in both the lab standards and the clinical strains!!
        - This was seen in both the original and rebasecalled data but not on the new sequencing kit (SK006?)
- The newer sequencing kit has a lot more noise than the others. Possibly due to the molecules moving faster through the pores in the new kit (a tradeoff between speed/yield and accuracy) and/or the metrichor base calling algorithm having more information with the old kits

Data:

Sequencing Runs:

- Quads study

    - Lab standard H1N1, H3N2, and FluB were sequenced on MiSeq, PacBio, and Ion Torrent. They were pooled individual as well as a mix of all three strains.
- MinION 4-20 and 4-21

    - Strains sequenced:
        - Lab standard H1N1
        - Lab standard H3N2
        - Lab standard FluB
- MinION 6-4

    - Strains sequenced:

        - clinical pH1N1 (a1)
        - clinical FluB 14 (f3)
        - clinical H3N2-70 (c1)
- MinION 8-13

    - Strains sequenced:

        - clinical pH1N1 (a1)
            - spike-in purified, non-size selected sample to compensate for loss of M/NS
        - clinical FluB 14 (f3) NP segment only
            - This was selected for because the rest of segments were well covered
        - clinical H3N2-90 (f1)
            - Different strain because??
        - FluB-HA (plasmid)
            - This was to see the error rate in the sequencer
    - These are all clinical samples. They were resequenced to obtain more data for consensus construction and variant calling.
- MinION 11-9

    - Strains sequenced

        - Lab standard H1N1
        - Lab standard FluB-PB1 (plasmid)
            - Done to resolve putative SSE
        - Clinical FluB 14
    - These were resequenced to resolve the errors in the consensus sequences of the previous runs.

MiSeq Sample Sheet:



L1 Norm comparisons:
MiSeq variants only
![alt text](miseq-minion-per-pos-manhattan-dists.pdf)
all positions

old metrichor vs new metrichor manhattan distance of clinical data

sk005 vs sk006 manhattan distance
positions H1N1-NS:586-599 were removed due to the deletion in the 4-20/21 sequence

Flu 11-9 Consensus



Mixed Infection Analysis
variants are present because this is only the 6-4 8-13 data. These variants have been resolved in the 11-9 data

Strain

Variants

Deletions

FluB-HA

1

10

FluB-MP

0

13

FluB-NA

0

6

FluB-NP

0

7

FluB-NS

0

1

FluB-PA

0

11

FluB-PB1

1

19

FluB-PB2

1

24

H1N1-HA

0

15

H1N1-MP

0

2

H1N1-NA

0

2

H1N1-NP

0

2

H1N1-NS

0

3

H1N1-PA

0

4

H1N1-PB1

0

9

H1N1-PB2

0

6

H3N2-70-HA

0

8

H3N2-70-MP

0

2

H3N2-70-NA

0

8

H3N2-70-NP

0

8

H3N2-70-NS

0

7

H3N2-70-PA

0

7

H3N2-70-PB1

0

17

H3N2-70-PB2

0

13

H3N2-90-HA

0

7

H3N2-90-MP

0

2

H3N2-90-NA

0

7

H3N2-90-NP

0

7

H3N2-90-NS

0

6

H3N2-90-PA

0

11

H3N2-90-PB1

0

17

H3N2-90-PB2

0

9

mixed infection coverage plot

lab standard coverage plot

11-9 coverage plot

Percent Consensus base

Variants between H3N2-70 and 90

Segment

Variations

HA

10

NA

10

NP

1

NS

2

MP

2

PA

2

PB1

6

PB2

7

Blast MinION data to get reference:

- megablast performed horribly. Others have reported success with blastn. That was better and the yields are as follows:
    - Influenza A virus (A/Egypt/42/2014(H1N1))
    - Influenza A virus (A/Singapore/H2013.778/2013(H3N2))
    - Influenza B virus (B/Nagasaki/13N070/2014)
- These are not what we were expecting so in order to be more precise a manually curated database was constructed
- Curated 2014-2015 Flu only database using the following parameters:

- Blast workflow was created as follows:
    - Blast raw MinION reads to the flu DB
    - Extract most frequent segments per subtype
        - Maybe change this to the top x% to assess mixed infections/reassortment?

        - Used this script:
    - Align to selected strains and generate consensus
        - Consensus sequences were very good when compared to the MiSeq data

        -
RefName RefLen  #Variants      #Deletions



FluB-NA        1401    0      6



FluB-NP        1683    0      7



FluB-PA        2181    0      10



FluB-PB1        2259    1      18



FluB-PB2        2313    2      25



H1N1-HA        1701    0      16



H1N1-NA        1410    0      2



H1N1-NP        1497    0      2



H1N1-PA        2151    0      4



H1N1-PB1        2274    0      9



H1N1-PB2        2280    0      6



H3N2-HA        1701    2      7



H3N2-MP        759    3      1



H3N2-NP        1497    1      7



H3N2-PB1        2274    4      16


H3N2-PB2        2280    8       10

        - Some segments aren’t listed because they were of different size - may be due to different coding sequences selected from the blast results?
    - blast the consensus again to see if there is any refinement

Data Analysis:

- MiSeq data were aligned to the CDS of the sequences that Bin provided

- The MiSeq consensus sequences were generated via this script:

- All data were then aligned to the MiSeq references and their subsequent consensus sequences were generated via the same script.
- Pac bio were aligned via blasr
    - /opt/bioinformatics-software/minion-software/blasr/alignment/bin/blasr Elodie_sidebar_reads_of_insert.fastq all-minion-flu-cds-only.fasta -sam -clipping soft -out pacbio-quad-cds-only.bam -nproc 40 -bestn 1
- MiSeq data were aligned using ‘bowtie2 –very-sensitive-local –X 2000 ‘
    - Insert size was increased beyond what was expected in order to allow for DIs
-  Ion data were aligned using ‘bowtie2 –very-sensitive-local’
- MinION data were aligned via bwa
    - /opt/bioinformatics-software/bwa-0.7.12/bwa mem -t 30 -x ont2d clinical-minion-refs.fasta pass.fasta
    - LAST wasn’t working as well

- The consensus sequences of each platform were compared using:












